In [1]:
import os
import time
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup
import os
import gspread
from google.oauth2 import service_account
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
import pandas as pd
current_directory = os.getcwd()
options = uc.ChromeOptions()
# options.add_argument(f"user-data-dir={profile_path}")
# options.add_argument(f"profile-directory={profile_name}")
driver = uc.Chrome(options=options)

In [2]:
driver.get("https://dungmori.com/backend/sql")


In [3]:
accounttxt_path = os.path.join(current_directory, 'account.txt')

with open(accounttxt_path, 'r', encoding='utf-8') as file:
    accounttxt = file.readlines()
    email = accounttxt[0]
    password = accounttxt[1]

In [4]:
# Điền địa chỉ email và tiếp tục
time.sleep(5)
email_input = WebDriverWait(driver, 1000).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="email"]'))
)
email_input.send_keys(email)

In [5]:
# Đợi và điền mật khẩu
password_input = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.XPATH, '//*[@id="password"]'))
)
password_input.send_keys(password)

In [6]:
time.sleep(2)
next_button = WebDriverWait(driver, 1000).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/section/div/div/div/form/div[5]/div/button')))
next_button.click()

In [7]:
time.sleep(2)
driver.get("https://dungmori.com/backend/sql")

In [8]:
group_query_path = os.path.join(current_directory, 'group info.sql')

with open(group_query_path, 'r', encoding='utf-8') as file:
    group_query = file.read()

In [9]:
time.sleep(2)
query_input = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div/div/textarea'))
)

In [10]:
driver.execute_script("arguments[0].value = arguments[1];", query_input, group_query)


In [11]:
time.sleep(2)
query_button = driver.find_element(By.XPATH,'//*[@id="sql_manager"]/button')
query_button.click()

In [12]:
time.sleep(5)

In [13]:
soup = BeautifulSoup(driver.page_source)

In [14]:
table = soup.find_all('table',class_='table table-bordered')


In [15]:
# table

In [16]:
headers = [header.get_text() for header in table[0].find_all('thead')[0].find_all('th')]


In [17]:
# headers

In [18]:
rows = []
for row in table[0].find_all('tbody')[0].find_all('tr'):
    cells = [cell.get_text() for cell in row.find_all('th')]
    rows.append(cells)

In [19]:
# rows

In [20]:
group_info_df = pd.DataFrame(rows, columns=headers)

In [21]:
# group_info_df

In [22]:
time.sleep(2)
driver.get('https://dungmori.com/backend/sql-school')

In [23]:
group_time_query_path = os.path.join(current_directory, 'group_time_tbl_count.sql')

with open(group_time_query_path, 'r', encoding='utf-8') as file:
    group_time_query = file.read()

In [24]:
time.sleep(2)
group_time_query_input = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div/div/textarea'))
)

In [25]:
driver.execute_script("arguments[0].value = arguments[1];", group_time_query_input,group_time_query)


In [26]:
time.sleep(2)
query_button = driver.find_element(By.XPATH,'//*[@id="sql_manager"]/button')
query_button.click()

In [27]:
time.sleep(5)

In [28]:
group_times_soup = BeautifulSoup(driver.page_source)

In [29]:
group_times_table = group_times_soup.find_all('table',class_='table table-bordered')


In [30]:
# group_times_table_headers

In [31]:
group_times_table_rows = []
for row in group_times_soup.find_all('tbody')[0].find_all('tr'):
    cells = [cell.get_text() for cell in row.find_all('th')]
    group_times_table_rows.append(cells)

In [32]:
group_times_df = pd.DataFrame(group_times_table_rows, columns=['group_id','session_count'])

In [33]:
# group_times_df

In [34]:
final_df = pd.merge(group_info_df,group_times_df,how = 'inner', on='group_id')

In [34]:
# final_df

In [35]:
change_type_columns = ['group_id','vip_session','shift_type','count_students','session_count']

In [37]:
final_df

,group_id,name,type,vip_level,vip_session,shift_type,shift_time,start_date,expired_date,count_students,session_count
0,1113,ONLINE VIP N3-32 DŨNG MORI (18/2/2022),vip15,N3,55,246,,2022-02-21,2022-06-27,14,1
1,1116,ONLINE VIP N4-43 DŨNG MORI(24/02/2022),vip15,N4,55,246,,2022-02-24,2022-07-12,16,1
2,1119,ONLINE VIP N4-42 Nguyễn Thị Thu Hà DŨNG MORI(...,vip15,N4,55,246,,2022-02-25,2022-07-06,14,3
3,1123,ONLIVE VIP N5-47 DŨNG MORI (02/03/2022),vip15,N5,60,246,,2022-03-02,2022-07-13,16,1
4,1126,ONLINE VIP N2-13 DŨNG MORI (01/03/2022),vip15,N2,13,357,19:00:00,2022-03-01,2022-07-05,16,55
...,...,...,...,...,...,...,...,...,...,...,...
1277,3273,N3-070824(1)-Nguyễn Thị Kim Chín DMR311 246,vip15,N3,55,246,19:00:00,2024-08-07,2025-01-08,17,1
1278,3275,FSOFT N2 (4)- 240724 - Phạm Thanh Nhàn 先生,vip15,N2,32,246,09:00:00,2024-07-24,2025-02-24,20,1
1279,3285,N4-050824(2)-Nguyễn Thị Thúy DMR244 (246),vip15,N4,55,246,19:00:00,2024-08-05,2025-02-12,17,2
1280,3293,N3 1:1 -080824- Bùi Thị Thanh Xuân DMR53,vip1,N3,45,2356,15:00:00,2024-08-08,2024-11-26,1,1


In [36]:
for column in change_type_columns:
    final_df[column] = final_df[column].astype(int)

ValueError: invalid literal for int() with base 10: ''

In [37]:
final_df['đợt chăm sóc'] = final_df.apply(
    lambda row: 1 if (
        (row['type'] == 'vip1' and row['session_count'] == 6) or
        (row['type'] == 'vip15' and (row['session_count'] ==6)) or
        (row['type'] == 'matgoc' and row['session_count'] == 3) or
        (row['type'] == 'captoc' and row['session_count'] == 12)
    ) else 2 if (
        (row['type'] == 'vip1' and row['session_count'] == 22) or
        (row['type'] == 'vip15' and row['session_count'] in range(28, 35)) or
        (row['type'] == 'matgoc' and row['session_count'] == 8) or
        (row['type'] == 'captoc' and row['session_count'] == 24)
    ) else 3 if (
        (row['type'] == 'vip1' and row['session_count'] == 40) or
        (row['type'] == 'vip15' and row['session_count'] in range(40, 51))
    ) else 0,
    axis=1
)


In [38]:
final_df.head()

,group_id,name,type,vip_level,vip_session,shift_type,shift_time,start_date,expired_date,count_students,session_count,đợt chăm sóc
0,2411,ONLINE VIP 500 N3- DŨNG MORI (T7/2024),vip500,N3,52,246,19:00:00,2024-02-19,2025-02-10,619,40,0
1,2412,ONLINE VIP 500 N2 - DŨNG MORI (T7/2024),vip500,N2,55,246,19:00:00,2024-02-19,2025-02-19,510,38,0
2,2426,ONLINE VIP 500 N1- DŨNG MORI (T7/2024),vip500,N1,52,246,19:00:00,2024-02-19,2025-02-10,324,37,0
3,2634,N1 - 160224 - NGUYỄN THỊ MỸ DMR175 (246; 18-20H),vip15,N1,55,246,18:00:00,2024-02-16,2024-08-09,20,55,0
4,2684,N4 - 160224 (1) - VŨ VĂN TOÀN DMR281 (246) (18...,vip15,N4,55,246,18:00:00,2024-02-16,2024-08-09,16,55,0


In [39]:
# filtered_df = final_df.loc[final_df['trang_thai'] == 1].sort_values(by = ['type','vip_level','session_count']).reset_index()

In [40]:
# filtered_df

In [38]:
# Đặt phạm vi (scope)
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

# Tải credentials từ file JSON
current_directory = os.getcwd()
file_path = os.path.join(current_directory, 'service_account_credentials.json')
creds = service_account.Credentials.from_service_account_file(file_path, scopes=scope)
client = gspread.authorize(creds)

# Mở Google Sheet bằng URL hoặc ID
spreadsheet = client.open_by_url('https://docs.google.com/spreadsheets/d/16c6xG0Rb9u_jkdGsjS7hU8wvDVdvnq2x7V2bhsMF42I/edit?gid=0#gid=0')
worksheet = spreadsheet.get_worksheet(0)  # Assuming you want to paste into the first worksheet
# Xóa tất cả dữ liệu trong worksheet
worksheet.clear()

# Ghi dữ liệu mới vào worksheet
set_with_dataframe(worksheet, final_df) 


In [ ]:
driver.quit()


In [47]:
# final_df.head().style.background_gradient(subset = ['session_count','đợt chăm sóc'],cmap='viridis')

,group_id,name,type,vip_level,vip_session,shift_type,shift_time,start_date,expired_date,count_students,session_count,đợt chăm sóc
0,2411,ONLINE VIP 500 N3- DŨNG MORI (T7/2024),vip500,N3,52,246,19:00:00,2024-02-19,2025-02-10,619,40,0
1,2412,ONLINE VIP 500 N2 - DŨNG MORI (T7/2024),vip500,N2,55,246,19:00:00,2024-02-19,2025-02-19,510,38,0
2,2426,ONLINE VIP 500 N1- DŨNG MORI (T7/2024),vip500,N1,52,246,19:00:00,2024-02-19,2025-02-10,324,37,0
3,2634,N1 - 160224 - NGUYỄN THỊ MỸ DMR175 (246; 18-20H),vip15,N1,55,246,18:00:00,2024-02-16,2024-08-09,20,55,0
4,2684,N4 - 160224 (1) - VŨ VĂN TOÀN DMR281 (246) (18h~20h),vip15,N4,55,246,18:00:00,2024-02-16,2024-08-09,16,55,0
